In [10]:
import re
import os
import sqlite3
import pandas as pd

In [33]:
conn = sqlite3.connect('rechtspraak.db')
c = conn.cursor()

In [14]:
pd.DataFrame(c.execute('''SELECT * FROM uitspraken_meta''').fetchmany(10))

,0,1,2,3
0,ECLI:NL:HR:1999:AA2833,"ECLI:NL:HR:1999:AA2833, Hoge Raad, 12-05-1999,...",-,2013-04-04T15:46:26+02:00
1,ECLI:NL:HR:1999:AA2862,"ECLI:NL:HR:1999:AA2862, Hoge Raad, 22-09-1999,...",-,2013-04-04T15:46:47+02:00
2,ECLI:NL:HR:1999:AA2863,"ECLI:NL:HR:1999:AA2863, Hoge Raad, 22-09-1999,...",-,2013-04-04T15:46:47+02:00
3,ECLI:NL:HR:1999:AA2867,"ECLI:NL:HR:1999:AA2867, Hoge Raad, 22-09-1999,...",-,2013-04-04T15:46:50+02:00
4,ECLI:NL:HR:1999:AA2868,"ECLI:NL:HR:1999:AA2868, Hoge Raad, 22-09-1999,...",-,2013-04-04T15:46:51+02:00
5,ECLI:NL:HR:1999:AA2869,"ECLI:NL:HR:1999:AA2869, Hoge Raad, 22-09-1999,...",-,2013-04-04T15:46:51+02:00
6,ECLI:NL:HR:1999:AA2871,"ECLI:NL:HR:1999:AA2871, Hoge Raad, 22-09-1999,...",-,2013-04-04T15:46:53+02:00
7,ECLI:NL:HR:1999:AA2872,"ECLI:NL:HR:1999:AA2872, Hoge Raad, 22-09-1999,...",-,2013-04-04T15:46:53+02:00
8,ECLI:NL:HR:1999:AA2873,"ECLI:NL:HR:1999:AA2873, Hoge Raad, 22-09-1999,...",-,2013-04-04T15:46:53+02:00
9,ECLI:NL:HR:1999:AA2874,"ECLI:NL:HR:1999:AA2874, Hoge Raad, 22-09-1999,...",-,2013-04-04T15:46:54+02:00


In [6]:
s = c.execute('SELECT COUNT(*) FROM UITSPRAKEN_META').fetchall()[0]
s

(29039,)

In [12]:
summary_counts = c.execute(''' SELECT * FROM (
                SELECT SUMMARY, COUNT(*) AS cnt
                FROM UITSPRAKEN_META
                GROUP BY SUMMARY
            ) ORDER BY CNT DESC
''') #.fetchall()
pd.DataFrame(summary_counts.fetchmany(20))

,0,1
0,-,5426
1,De Hoge Raad heeft het beroep in cassatie afge...,860
2,HR: 81.1 RO.,756
3,HR verklaart het beroep in cassatie n-o met to...,401
4,HR verklaart het beroep in cassatie n-o.,288
5,HR: art. 81.1 RO.,253
6,De Hoge Raad heeft het beroep in cassatie niet...,239
7,De Hoge Raad heeft het beroep in cassatie afge...,203
8,De Hoge Raad heeft het beroep in cassatie niet...,192
9,De Hoge Raad heeft het beroep in cassatie afge...,179


In [21]:
lenid_counts = c.execute(''' SELECT length(id), count(*) FROM UITSPRAKEN_META group by length(id)
''')
pd.DataFrame(lenid_counts.fetchall())

,0,1
0,17,37
1,18,356
2,19,2524
3,20,4005
4,22,22117


In [28]:
def nr_of_fields_id(text):
    return len(text.split(':'))

def year_from_id(text):
    return text.split(':')[3]

def instantie_from_id(text):
    return text.split(':')[2]

conn.create_function('nr_of_fields_id', 1, nr_of_fields_id)
conn.create_function('year_from_id', 1, year_from_id)
conn.create_function('instantie_from_id', 1, instantie_from_id)
conn.commit()

In [23]:
nrfields_counts = c.execute(''' SELECT nr_of_fields_id(id), count(*) FROM UITSPRAKEN_META group by nr_of_fields_id(id)
''')
pd.DataFrame(nrfields_counts.fetchall())

,0,1
0,5,29039


In [27]:
instantie_counts = c.execute(''' SELECT instantie_from_id(id), count(*) 
                FROM UITSPRAKEN_META group by instantie_from_id(id) 
''')
pd.DataFrame(instantie_counts.fetchall())

,0,1
0,HR,29039


In [26]:
year_counts = c.execute(''' SELECT year_from_id(id), count(*) 
                FROM UITSPRAKEN_META group by year_from_id(id) 
                order by year_from_id(id) 
''')
pd.DataFrame(year_counts.fetchall())

,0,1
0,1968,1
1,1984,1
2,1988,1
3,1991,2
4,1992,6
5,1993,2
6,1994,84
7,1995,332
8,1996,354
9,1997,369


In [30]:
#How many links do we have?
c.execute('''select count(distinct id), count(*) from uitspraken_links''').fetchall()

[(1778, 3000)]

In [36]:
#How many of those are to other HR cases?
c.execute('''select count(distinct l.id), count(*) 
            from uitspraken_links l join 
                ( select id from uitspraken_meta ) s
                on s.id = l.reference
        ''').fetchall()

[(1645, 2497)]

In [37]:
c.close()